In [ ]:
# !pip install splinter
# !pip install webdriver-manager
# reference blog: https://www.codecademy.com/resources/blog/web-scraping-python-beautiful-soup-mlb-stats/

In [1]:
import requests
import os
import re
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Set up Splinter Executable path
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\Admin\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


In [3]:
# Visit espn salaries page
url = 'http://www.espn.com/nba/salaries'
browser.visit(url)
# Search all elements with the tag div with attribute list_text, then wait 1 second before searching components
browser.is_element_present_by_css('div.list_text', wait_time=1)

False

In [4]:
# set up html parser
html = browser.html
soup = bs(html, 'html.parser')

In [5]:
# create the dataframe
header = soup.find('tr', class_='colhead')
columns = [col.get_text() for col in header.find_all('td')]
columns.append('season')
salaries_df = pd.DataFrame(columns = columns)
salaries_df

,RK,NAME,TEAM,SALARY,season


In [8]:
# max_page_num = soup.find('div', class_ = 'page-numbers').text
# last_page = max_page_num[-2:]
# get a list of the years from drop down menu
menu = soup.select('option[value]')
years = [year.text for year in menu[1:]]
i = 0
# while loop to keep the loop running as long as there is an attribute to click
while True:
    # Reinitialize soup for scrapping
    html = browser.html
    soup = bs(html, 'html.parser')
    
    # grab all the players data (RK, NAME, TEAM, SALARY) on the current page
    players = soup.find_all('tr', attrs={'class':re.compile('row')})
    # finds the year that we are currently on
    menu = soup.select('option[selected]')
    year = [menu.text for menu in menu]
    # looping through all of the soup players data
    for player in players:       
        # get info and salaries and append year
        salary = [info.get_text() for info in player.find_all('td')]
        salary.append(year[0])
        print(salary)
        
        # create a temporary df
        temp_df = pd.DataFrame(salary).transpose()
        temp_df.columns = columns
        salaries_df = pd.concat([salaries_df, temp_df], ignore_index = True)
    try:
        # find next button and press
        next_button =  browser.find_by_tag('div[class = "jcarousel-next"]')
        next_button.click()
    except:
        print(f"Done Scrapping: {year}")  
        try:
            i+=1
            print(i)
            element = browser.find_by_tag('select[class = "tablesm"]').click()
            browser.find_by_text(years[i]).click()
        except:
            print("Done with Scrapping")
            break

['1', 'Stephen Curry, PG', 'Golden State Warriors', '$45,780,966', '2021-2022']
['2', 'James Harden, SG', 'Brooklyn Nets', '$44,310,840', '2021-2022']
['3', 'John Wall, PG', 'Houston Rockets', '$44,310,840', '2021-2022']
['4', 'Russell Westbrook, PG', 'Los Angeles Lakers', '$44,211,146', '2021-2022']
['5', 'Kevin Durant, PF', 'Brooklyn Nets', '$42,018,900', '2021-2022']
['6', 'LeBron James, SF', 'Los Angeles Lakers', '$41,180,544', '2021-2022']
['7', 'Giannis Antetokounmpo, PF', 'Milwaukee Bucks', '$39,344,970', '2021-2022']
['8', 'Damian Lillard, PG', 'Portland Trail Blazers', '$39,344,900', '2021-2022']
['9', 'Paul George, SG', 'LA Clippers', '$39,344,900', '2021-2022']
['10', 'Klay Thompson, SG', 'Golden State Warriors', '$37,980,720', '2021-2022']
['11', 'Kemba Walker, PG', 'New York Knicks', '$36,016,200', '2021-2022']
['12', 'Jimmy Butler, SF', 'Miami Heat', '$36,016,200', '2021-2022']
['13', 'Kawhi Leonard, SF', 'LA Clippers', '$36,016,200', '2021-2022']
['14', 'Tobias Harris, P

['121', 'Reggie Jackson, PG', 'LA Clippers', '$10,384,500', '2021-2022']
['122', 'Anthony Edwards, SF', 'Minnesota Timberwolves', '$10,245,480', '2021-2022']
['123', 'Al-Farouq Aminu, PF', 'San Antonio Spurs', '$10,183,800', '2021-2022']
['124', 'Luka Doncic, PG', 'Dallas Mavericks', '$10,174,391', '2021-2022']
['125', 'Cade Cunningham, PG', 'Detroit Pistons', '$10,050,120', '2021-2022']
['126', 'Danny Green, SF', 'Philadelphia 76ers', '$10,000,000', '2021-2022']
['127', 'Tomas Satoransky, SG', 'New Orleans Pelicans', '$10,000,000', '2021-2022']
['128', 'Kyle Anderson, SF', 'Memphis Grizzlies', '$9,937,150', '2021-2022']
['129', 'DeAndre Jordan, C', 'Los Angeles Lakers', '$9,881,598', '2021-2022']
['130', 'Montrezl Harrell, C', 'Washington Wizards', '$9,720,900', '2021-2022']
['131', 'Derrick Favors, C', 'Oklahoma City Thunder', '$9,720,900', '2021-2022']
['132', 'Tristan Thompson, C', 'Sacramento Kings', '$9,720,900', '2021-2022']
['133', 'Jae Crowder, PF', 'Phoenix Suns', '$9,720,900

['241', 'Dorian Finney-Smith, PF', 'Dallas Mavericks', '$4,000,000', '2021-2022']
['242', 'Gorgui Dieng, C', 'Atlanta Hawks', '$4,000,000', '2021-2022']
['243', 'Aaron Holiday, PG', 'Washington Wizards', '$3,980,551', '2021-2022']
['244', 'Joshua Primo, SG', 'San Antonio Spurs', '$3,946,800', '2021-2022']
['245', 'Jake Layman, SF', 'Minnesota Timberwolves', '$3,940,185', '2021-2022']
['246', 'Anfernee Simons, SG', 'Portland Trail Blazers', '$3,938,818', '2021-2022']
['247', 'Justise Winslow, PF', 'LA Clippers', '$3,902,439', '2021-2022']
['248', 'Danuel House Jr., SF', 'Houston Rockets', '$3,894,000', '2021-2022']
['249', 'Kira Lewis Jr., PG', 'New Orleans Pelicans', '$3,822,240', '2021-2022']
['250', 'Romeo Langford, SG', 'Boston Celtics', '$3,804,360', '2021-2022']
['251', 'Landry Shamet, SG', 'Phoenix Suns', '$3,768,342', '2021-2022']
['252', 'Chris Duarte, SG', 'Indiana Pacers', '$3,749,400', '2021-2022']
['253', 'Alex Len, C', 'Sacramento Kings', '$3,732,000', '2021-2022']
['254',

['361', 'Matt Thomas, SG', 'Chicago Bulls', '$1,782,621', '2021-2022']
['362', 'Bruno Fernando, F', 'Boston Celtics', '$1,782,621', '2021-2022']
['363', 'Yuta Watanabe, SF', 'Toronto Raptors', '$1,762,796', '2021-2022']
['364', 'Drew Eubanks, PF', 'San Antonio Spurs', '$1,762,796', '2021-2022']
['365', 'Chimezie Metu, PF', 'Sacramento Kings', '$1,762,796', '2021-2022']
['366', 'Khyri Thomas, SG', 'Houston Rockets', '$1,762,796', '2021-2022']
['367', 'Alize Johnson, F', 'Chicago Bulls', '$1,762,796', '2021-2022']
['368', 'Wenyen Gabriel, PF', 'New Orleans Pelicans', '$1,762,796', '2021-2022']
['369', 'Charlie Brown Jr., F', 'Oklahoma City Thunder', '$1,701,593', '2021-2022']
['370', 'Paul Watson, SG', 'Oklahoma City Thunder', '$1,701,593', '2021-2022']
['371', 'DaQuan Jeffries, SG', 'San Antonio Spurs', '$1,701,593', '2021-2022']
['372', 'Oshae Brissett, SF', 'Indiana Pacers', '$1,701,593', '2021-2022']
['373', 'Kelan Martin, SG', 'Indiana Pacers', '$1,701,593', '2021-2022']
['374', 'Ju

['441', 'Lamar Stevens, F', 'Cleveland Cavaliers', '$1,517,981', '2021-2022']
['442', 'Naji Marshall, SF', 'New Orleans Pelicans', '$1,517,981', '2021-2022']
['443', 'Isaiah Todd, PF', 'Washington Wizards', '$1,500,000', '2021-2022']
['444', 'Omer Yurtseven, C', 'Miami Heat', '$1,489,065', '2021-2022']
['445', 'Saben Lee, PG', 'Detroit Pistons', '$1,489,065', '2021-2022']
['446', 'JT Thor, PF', 'Charlotte Hornets', '$1,250,000', '2021-2022']
['447', 'Jason Preston, PG', 'LA Clippers', '$1,062,303', '2021-2022']
['448', 'Isaiah Livers, PF', 'Detroit Pistons', '$1,057,260', '2021-2022']
['449', 'Dalano Banton, PG', 'Toronto Raptors', '$925,258', '2021-2022']
['450', 'Jared Butler, SG', 'Utah Jazz', '$925,258', '2021-2022']
['451', 'Jock Landale, C', 'San Antonio Spurs', '$925,258', '2021-2022']
['452', 'Greg Brown III, PF', 'Portland Trail Blazers', '$925,258', '2021-2022']
['453', 'Miles McBride, PG', 'New York Knicks', '$925,258', '2021-2022']
['454', 'Ayo Dosunmu, SG', 'Chicago Bulls'

...

['41', 'Eddie Jones, SG', 'Miami Heat', '$8,960,000', '2000-2001']
['42', 'Brian Grant, PF', 'Miami Heat', '$8,897,000', '2000-2001']
['43', 'Tim Thomas, PF', 'Milwaukee Bucks', '$8,507,000', '2000-2001']
['44', 'Michael Finley, SG', 'Dallas Mavericks', '$8,400,000', '2000-2001']
['45', 'Shawn Bradley, C', 'Dallas Mavericks', '$8,370,000', '2000-2001']
['46', 'Terrell Brandon, PG', 'Minnesota Timberwolves', '$8,325,000', '2000-2001']
['47', 'Nick Van Exel, PG', 'Denver Nuggets', '$8,184,000', '2000-2001']
['48', 'Glenn Robinson, SF', 'Milwaukee Bucks', '$8,120,000', '2000-2001']
['49', 'Steve Smith, SG', 'Portland Trail Blazers', '$8,100,000', '2000-2001']
['50', 'Elden Campbell, C', 'Charlotte Hornets', '$8,050,000', '2000-2001']
['51', 'Derrick Coleman, PF', 'Charlotte Hornets', '$8,040,000', '2000-2001']
['52', 'Avery Johnson, PG', 'San Antonio Spurs', '$8,000,000', '2000-2001']
['53', 'Horace Grant, C', 'Los Angeles Lakers', '$7,843,000', '2000-2001']
['54', 'Theo Ratliff, C', 'Phi

['161', 'Cedric Henderson, F', 'Cleveland Cavaliers', '$2,700,000', '2000-2001']
['162', 'Kevin Willis, C', 'Denver Nuggets', '$2,700,000', '2000-2001']
['163', 'Lawrence Funderburke, PF', 'Sacramento Kings', '$2,700,000', '2000-2001']
['164', 'Antawn Jamison, PF', 'Golden State Warriors', '$2,678,000', '2000-2001']
['165', 'Lindsey Hunter, SG', 'Milwaukee Bucks', '$2,650,000', '2000-2001']
['166', 'Lamar Odom, PF', 'Los Angeles Clippers', '$2,628,000', '2000-2001']
['167', 'Malik Rose, PF', 'San Antonio Spurs', '$2,570,000', '2000-2001']
['168', 'Marcus Fizer, F', 'Chicago Bulls', '$2,562,000', '2000-2001']
['169', 'Dragan Tarlac, C', 'Chicago Bulls', '$2,500,000', '2000-2001']
['170', 'Vince Carter, G', 'Toronto Raptors', '$2,425,000', '2000-2001']
['171', 'Jonathan Bender, SF', 'Indiana Pacers', '$2,380,000', '2000-2001']
['172', 'Chris Whitney, PG', 'Washington Wizards', '$2,370,000', '2000-2001']
['173', 'Mike Miller, SG', 'Orlando Magic', '$2,320,000', '2000-2001']
['174', 'Othel

['281', 'Fred Hoiberg, SG', 'Chicago Bulls', '$880,000', '2000-2001']
['282', 'Tyronn Lue, PG', 'Los Angeles Lakers', '$865,000', '2000-2001']
['283', 'Devean George, SF', 'Los Angeles Lakers', '$849,000', '2000-2001']
['284', 'Felipe Lopez, G', 'Minnesota Timberwolves', '$831,000', '2000-2001']
['285', 'DeShawn Stevenson, SG', 'Utah Jazz', '$828,000', '2000-2001']
['286', 'John Crotty, G', 'Utah Jazz', '$800,000', '2000-2001']
['287', 'Don Maclean, F', 'Miami Heat', '$798,000', '2000-2001']
['288', 'Al Harrington, PF', 'Indiana Pacers', '$797,000', '2000-2001']
['289', 'Dalibor Bagaric, C', 'Chicago Bulls', '$795,000', '2000-2001']
['290', 'Tim James, F', 'Charlotte Hornets', '$783,000', '2000-2001']
['291', 'Jake Tsakalidis, C', 'Phoenix Suns', '$763,000', '2000-2001']
['292', 'Vonteego Cummings, PG', 'Golden State Warriors', '$757,000', '2000-2001']
['293', 'Hubert Davis, SG', 'Washington Wizards', '$752,000', '2000-2001']
['294', 'Corey Benjamin, G', 'Chicago Bulls', '$744,000', '2

['41', 'Donyell Marshall, PF', 'Golden State Warriors', '$5,250,000', '1999-2000']
['42', 'Marcus Camby, C', 'New York Knicks', '$5,250,000', '1999-2000']
['43', 'Greg Ostertag, C', 'Utah Jazz', '$5,200,000', '1999-2000']
['44', 'Luc Longley, C', 'Phoenix Suns', '$5,175,000', '1999-2000']
['45', 'P.J. Brown, PF', 'Miami Heat', '$5,124,000', '1999-2000']
['46', "Jermaine O'Neal, C", 'Portland Trail Blazers', '$5,075,000', '1999-2000']
['47', 'Erick Dampier, C', 'Golden State Warriors', '$4,988,000', '1999-2000']
['48', 'Wesley Person, SF', 'Cleveland Cavaliers', '$4,900,000', '1999-2000']
['49', 'Isaac Austin, C', 'Washington Wizards', '$4,819,000', '1999-2000']
['50', 'Loy Vaught, F', 'Detroit Pistons', '$4,810,000', '1999-2000']
['51', 'Robert Horry, PF', 'Los Angeles Lakers', '$4,800,000', '1999-2000']
['52', 'Derrick McKey, F', 'Indiana Pacers', '$4,800,000', '1999-2000']
['53', 'Jim Mcilvaine, C', 'New Jersey Nets', '$4,800,000', '1999-2000']
['54', 'Jerry Stackhouse, SF', 'Detroit

Done Scrapping: ['1999-2000']
23
Done with Scrapping


In [9]:
salaries_df

,RK,NAME,TEAM,SALARY,season
0,1,"Stephen Curry, PG",Golden State Warriors,"$45,780,966",2021-2022
1,2,"James Harden, SG",Brooklyn Nets,"$44,310,840",2021-2022
2,3,"John Wall, PG",Houston Rockets,"$44,310,840",2021-2022
3,4,"Russell Westbrook, PG",Los Angeles Lakers,"$44,211,146",2021-2022
4,5,"Kevin Durant, PF",Brooklyn Nets,"$42,018,900",2021-2022
...,...,...,...,...,...
10631,153,"Al Harrington, PF",Indiana Pacers,"$745,000",1999-2000
10632,154,"Tim James, F",Miami Heat,"$728,000",1999-2000
10633,155,"Jumaine Jones, SF",Philadelphia 76ers,"$684,000",1999-2000
10634,156,"Scott Padgett, PF",Utah Jazz,"$679,000",1999-2000


In [10]:
browser.quit()

In [11]:
salaries_df.to_csv('1999-2022 Salaries.csv', index = False)